In [19]:
import pandas as pd
from functions import to_array, factorize, PCA2d, PCA3d

import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook"



# PCA(Principal Component Analysis)

*PCA (Principal Component Analysis) for genotype data is a commonly used method in genetics research to reduce the dimensionality of genotype data and visualize the relationships between individuals or populations.*

*In this context, PCA is applied to a matrix of genotype data, where each row represents an individual and each column represents a SNP (Single Nucleotide Polymorphism). PCA finds the main patterns of genetic variation in the data and creates a set of new variables, called principal components, that capture the majority of the variation in the original data.*

*These principal components can then be plotted in a two- or three-dimensional space, allowing us to visualize the relationships between individuals or populations based on their genetic similarity or differences. This can help us identify clusters of individuals with similar genetic profiles, detect population substructure or admixture, and even identify potential genetic outliers or outliers in the data. PCA is a powerful tool for exploring genotype data and has many applications in genetics research.*

In [20]:
org_data = pd.read_excel('./data/data_Ali.xlsx', engine='openpyxl')

In [21]:
p_val = pd.read_csv('./data/R_stat_p.csv')
p_val.rename(columns={p_val.columns[0]: 'rs'}, inplace=True)
# p_val

In [ ]:
# print('Hi there')

In [23]:
group = org_data.Status


In [42]:
n = 100
PCA2d(org_data[p_val.rs[:n]], group, title = f'2D PCA<br>({n} SNPs & {group.shape[0]} samples).')

In [38]:
n = 100
# PCA3d(org_data[p_val.rs[:n]], group, title = f'3D PCA<br>({n} SNPs & {group.shape[0]} samples)')

In [35]:
n = 80
PCA2d(org_data[p_val.rs[:n]], group, title = f'2D PCA<br>({n} SNPs & {group.shape[0]} samples)')

In [33]:
n = 80
PCA3d(org_data[p_val.rs[:n]], group, title = f'3D PCA<br>({n} SNPs & {group.shape[0]} samples)')

In [34]:
import numpy as np
import pandas as pd

from scipy.stats import chi2_contingency
from scipy.stats.contingency import odds_ratio

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import seaborn as sns
import matplotlib.pyplot as plt

from adjustText import adjust_text

from sklearn.impute import KNNImputer
from sklearn.decomposition import PCA
from sklearn.preprocessing import OrdinalEncoder

from ast import literal_eval


def fill(arr):
    imputer = KNNImputer(n_neighbors=10)
    print(arr.shape)
    d = imputer.fit_transform(arr)
    print(d.shape)
    return d


def fill_numerical(Data):
    filled = fill(Data.copy())
    # print(len(Data.columns))
    # print(filled.shape)
    # print(Data.copy().shape)
    return pd.DataFrame(filled, columns=Data.columns)


def fill_categorical(Cat_data):
    # encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
    encoder = OrdinalEncoder()

    encoded_data = encoder.fit_transform(Cat_data.values)

    encoded_data[encoded_data == -1] = np.nan
    data = fill(encoded_data)

    encoded_data = np.round(data)

    # Inverse transform the encoded data to get the original data
    df_decoded = pd.DataFrame(encoder.inverse_transform(encoded_data), columns=Cat_data.columns)

    return df_decoded

 



def to_array(df):
    def convert_to_array(col):
        """
        Convert genotypic information to an array.
        """
        w, m = col.name[-3:].split('>')
        wild, heterozygous, mutant = w+w, w+m, m+m
    
        replace_values = {
            wild : '[0, 0]',
            heterozygous : '[0, 1]',
            mutant : '[1, 1]'
        }

        res = col.replace(replace_values)
        try:
            res = res.apply(lambda x: np.array(literal_eval(str(x))))
        except:
            print(res.unique(), col.name)
            
        return res
        
    result = df.apply(convert_to_array)

    return result

def factorize(df):
    df = df.apply(lambda x: pd.factorize(x)[0])
    return df


def PCA3d(data, groups, title='3D PCA'):
    data = factorize(data)
    X_reduced = PCA(n_components=3).fit_transform(data)
    # Create a dataframe with the reduced data
    df = pd.DataFrame(X_reduced, columns=['PC1', 'PC2', 'PC3'])
    df['Groups'] = groups
    # Create the scatter plot with Plotly
    fig = px.scatter_3d(df, x='PC1', y='PC2', z='PC3', color='Groups', 
                        symbol='Groups', opacity=0.9,
                        width=800, height=800)

    # Set the title and axis labels
    fig.update_layout(title=title, 
                      template='plotly_white', 
                    scene=dict(xaxis_title='1st eigenvector', yaxis_title='2nd eigenvector', 
                                zaxis_title='3rd eigenvector'))
    fig.update_traces(marker_size=5)
    fig.update_coloraxes(showscale=False)
    return fig

def PCA2d(data, groups, title = '2D PCA'):
    data = factorize(data)

    layout = go.Layout(
        plot_bgcolor="#FFFFFF",
        barmode="stack",
        xaxis=dict(
            linecolor="#BCCCDC",
        ),
        yaxis=dict(
            linecolor="#BCCCDC"
        )
    )


    X_reduced = PCA(n_components=2).fit_transform(data)
    # Create a dataframe with the reduced data
    df = pd.DataFrame(X_reduced, columns=['PC1', 'PC2'])
    df['Groups'] = groups
    # Create the scatter plot with Plotly
    fig = px.scatter(df[::-1], x='PC1', y='PC2', color='Groups',opacity=1,
                        width=700, height=500)

    # Set the title and axis labels
    fig.update_layout(title=title, 
                    #   template='plotly_white', 
                    #   scene=dict(xaxis_title='1st eigenvector', yaxis_title='2nd eigenvector', 
                    #              zaxis_title='3rd eigenvector')
                                )
    # fig.update_layout(template='plotly_white', )
    fig.update_traces(marker_size=8)
    fig.update_coloraxes(showscale=False)
    return fig



In [11]:
### Old version(Python stat)

In [12]:
# p_val = pd.read_csv('./data/R_stat_p.csv').iloc[:, :3]
# p_val.columns = ['rs', 'p_value_allele','p_value_genotype']

In [13]:
# rs = p_val[(p_val.p_value_genotype<0.05)&(p_val.p_value_allele<0.05)].rs.values

In [14]:
# print('SNPs with the P-value less than 0.05(both genotype and allele, total:13):',*rs, sep='\n    ')

### SNPs with the lowest P values based on **genotype** analysis.

In [15]:
# sorted_rs_allele = p_val.sort_values(by='p_value_allele').rs.values
# sorted_rs_gen = p_val.sort_values(by='p_value_genotype').rs.values
# rs_lower5 = p_val[(p_val.p_value_genotype<0.05)&(p_val.p_value_allele<0.05)].rs.values
# p_val.sort_values(by=['p_value_genotype','p_value_allele']).head(10)

In [16]:
# n = 8
# PCA2d(data[sorted_rs_gen[:n]], group, title = f'3D PCA<br>({n} SNPs & 80 samples)')

In [17]:
# n = 8
# PCA3d(data[sorted_rs_gen[:n]], group, title = f'3D PCA<br>({n} SNPs & 80 samples)')

In [18]:
# n = 70
# PCA2d(data[sorted_rs_gen[:n]], group, title = f'3D PCA<br>({n} SNPs & 80 samples)')

In [19]:
# n = 70
# PCA3d(data[sorted_rs_gen[:n]], group, title = f'3D PCA<br>({n} SNPs & 80 samples)')

In [20]:
### SNPs with the lowest P values based on **allele** analysis.

In [21]:
# p_val.sort_values(by='p_value_allele').head(10)

In [22]:
# n = 8
# PCA2d(data[sorted_rs_allele[:n]], group, title = f'3D PCA<br>({n} SNPs & 80 samples)')

In [23]:
# n = 8
# PCA3d(data[sorted_rs_allele[:n]], group, title = f'3D PCA<br>({n} SNPs & 80 samples)')

In [24]:
# n = 70
# PCA2d(data[sorted_rs_allele[:n]], group, title = f'3D PCA<br>({n} SNPs & 80 samples)')

In [25]:
# n = 70
# PCA3d(data[sorted_rs_allele[:n]], group, title = f'3D PCA<br>({n} SNPs & 80 samples)')